In [1]:
from heapdict import heapdict
import numpy as np
import networkx as nx
import pickle
import random
import math
import torch
from os import path
import torch.nn.functional as F
import torch.nn as nn


In [2]:
def pickle_save(data,file_name):
    if path.exists(file_name):
        raise BaseException('file already exists')

    with open(file_name,'wb') as f:
        pickle.dump(data , f)

def pickle_load(file_name):
    with open(file_name,'rb') as f:
        return pickle.load(f)
def density2regD(n  , density):
    return math.floor(n*density)

def density_to_edge_ba(n , p):
    return math.ceil( p *n  /2)


In [3]:
def validation_graph_gen(n , p , num = 100, graph_type = 'er'):
    np.random.seed(20000420)
    random.seed(20000420)
    validation_graph = []
    for i in range(num):
        if graph_type == 'er':
            g = nx.erdos_renyi_graph(n , p)
            validation_graph.append(g)
        elif graph_type == 'ba':
            m = density_to_edge_ba(n , p)
            g = nx.barabasi_albert_graph(n , m)
            validation_graph.append(g)
        elif graph_type == 'reg':
            d = density2regD(n , p)
            g = nx.random_regular_graph(n = n , d = d)
            validation_graph.append(g)
        elif graph_type == 'pow':
            m = density_to_edge_ba(n , p)
            g = nx.powerlaw_cluster_graph(n = n , m = m , p = 0.25)
            validation_graph.append(g)
    return validation_graph

In [4]:
def mvc_bb(graph , UB = 9999999 , C = []):    
    if len(graph.edges()) == 0:
        return C
    LB = 0 #lower bound
    # if lower bound add current cover number larger than upper bound, it means there is impossible to find the better result
    if len(C) + LB >= UB: 
        return [i for i in range(UB+1)]
    
    degree_list = nx.degree(graph)
    v , d = max(degree_list , key = lambda a : a[1])
    #use branch and bound to find out the mvc result
    #C1 will include all neighbor node of v
    #C2 will only include v
    C1 = C[:] 
    C2 = C[:]
    graph_1 = graph.copy()
    C1.extend(list(graph.neighbors(v)))
    graph_1.remove_nodes_from(C1)
    C1= mvc_bb(graph_1 , UB , C1)

    C2.append(v)
    graph_2 = graph.copy()
    graph_2.remove_node(v)
    C2 = mvc_bb(graph_2 , min(UB , len(C1)) , C2 )

    if len(C1)>len(C2):
        return C2
    else:
        return C1

In [5]:
data, opt = pickle_load("/workspace/Synthetic_graph/Training_graph_50_withOPT.pkl")

In [6]:
G1 = data[0]
G2 = data[1]


In [7]:
A_np = nx.adjacency_matrix(G1).todense()
A_np = np.array(A_np)
A_flatten = A_np.flatten()
# 將 NumPy 陣列轉換為 PyTorch 張量
A_torch = torch.tensor(A_flatten)

# 將張量移動到 GPU 上（假設你有可用的 CUDA GPU）
if torch.cuda.is_available():
    A_torch = A_torch.to('cuda')

In [8]:
A2_np = nx.adjacency_matrix(G2).todense()
A2_np = np.array(A2_np)
A2_flatten = A2_np.flatten()
# 將 NumPy 陣列轉換為 PyTorch 張量
A2_torch = torch.tensor(A2_flatten)


# 將張量移動到 GPU 上（假設你有可用的 CUDA GPU）
if torch.cuda.is_available():
    A2_torch = A2_torch.to('cuda')

In [9]:
cos = nn.CosineSimilarity(dim=1)
A_torch = A_torch.float().unsqueeze(0)
A2_torch = A2_torch.float().unsqueeze(0)
print(cos(A_torch, A2_torch))

tensor([0.6489], device='cuda:0')


In [12]:
from numpy.linalg import norm
cosine_similarity = np.dot(A_flatten, A2_flatten) / (norm(A_flatten) * norm(A2_flatten))
print(cosine_similarity)

0.6488767461182359


In [13]:
training_solution = []
for i in data :
    training_solution.append(mvc_bb(i))

In [15]:
pickle_save((data,training_solution) , "/workspace/Synthetic_graph/Training_graph_50_withOPT.pkl")

In [ ]:
training_graph = []
opt = []
pro = []
for i in range(1000):
    p = random.uniform(0.25,0.75)
    G = nx.erdos_renyi_graph(50, p)
    MVC = len(mvc_bb(G))
    print(f"i = {i}, p = {p}, MVC = {MVC}")
    training_graph.append(G)
    opt.append(MVC)
    pro.append(p)
pickle_save((training_graph,opt,pro) , 'workspace/Testing_graph_1000_withOPTPRO.pkl')

In [16]:
valid_graph = []
opt = []
pro = []
for i in range(200):
    p = random.uniform(0.25,0.75)
    G = nx.erdos_renyi_graph(50, p)
    MVC = len(mvc_bb(G))
    print(f"i = {i}, p = {p}, MVC = {MVC}")
    valid_graph.append(G)
    opt.append(MVC)
    pro.append(p)
pickle_save((valid_graph,opt,pro) , 'workspace/Validation_graph_200_withOPTPRO.pkl')

i = 0, p = 0.6952651275933455, MVC = 45
i = 1, p = 0.5927124962222894, MVC = 45
i = 2, p = 0.3257294073954657, MVC = 39
i = 3, p = 0.5833821011022489, MVC = 44
i = 4, p = 0.3081408098387335, MVC = 40
i = 5, p = 0.3232772802619697, MVC = 39
i = 6, p = 0.26010065255517306, MVC = 38
i = 7, p = 0.26155676581699416, MVC = 38
i = 8, p = 0.2890938460791986, MVC = 38
i = 9, p = 0.5976792598535426, MVC = 44
i = 10, p = 0.2919590391520179, MVC = 38
i = 11, p = 0.6240768030130797, MVC = 44
i = 12, p = 0.48403042542701713, MVC = 42
i = 13, p = 0.7415302708550214, MVC = 45
i = 14, p = 0.3056030146735697, MVC = 38
i = 15, p = 0.27381396805690517, MVC = 38
i = 16, p = 0.36854803978345974, MVC = 40
i = 17, p = 0.40603781446772863, MVC = 41
i = 18, p = 0.44994550739096856, MVC = 42
i = 19, p = 0.33232145167903293, MVC = 39
i = 20, p = 0.654364748850738, MVC = 45
i = 21, p = 0.6400252829284272, MVC = 44
i = 22, p = 0.35273876572181145, MVC = 40
i = 23, p = 0.4501795460280074, MVC = 42
i = 24, p = 0.6568

In [4]:
def greedy2approx(graph):
    """this function is an approximate method"""
    covered_edge = 0
    num_edge = len(graph.edges())
    hd = heapdict()
    degree = nx.degree(graph)
    for v , d in degree:
        hd[v] = -d
        
    select_nodes = set()
    
    while covered_edge < num_edge:
        cur_v , cur_deg = hd.popitem()
        select_nodes.add(cur_v)
        covered_edge += -(cur_deg)
        for u in graph.neighbors(cur_v):
            if u not in select_nodes:
                hd[u] += 1
    return select_nodes

In [7]:
Test = validation_graph_gen(n= 1000,p = 0.15, num = 1, graph_type='er')

In [8]:
Z = greedy2approx(Test[0])

In [23]:
er_01 = validation_graph_gen(n= 50,p = 0.1, num = 100, graph_type='er')
er_02 = validation_graph_gen(n= 50,p = 0.2, num = 100, graph_type='er')
er_03 = validation_graph_gen(n= 50,p = 0.3, num = 100, graph_type='er')
er_04 = validation_graph_gen(n= 50,p = 0.4, num = 100, graph_type='er')
er_05 = validation_graph_gen(n= 50,p = 0.5, num = 100, graph_type='er')
er_06 = validation_graph_gen(n= 50,p = 0.6, num = 100, graph_type='er')
er_07 = validation_graph_gen(n= 50,p = 0.7, num = 100, graph_type='er')
er_08 = validation_graph_gen(n= 50,p = 0.8, num = 100, graph_type='er')
er_09 = validation_graph_gen(n= 50,p = 0.9, num = 100, graph_type='er')

In [ ]:
ans = []
for i in range(100):
    print(i)
    ans.append(len(mvc_bb(er_01[i])))

In [27]:
pickle_save((er_01,ans),"workspace/Synthetic_graph/er_01_MVC_ANS.pkl")

In [ ]:
ans1 = []
for i in range(100):
    print(i)
    ans1.append(len(mvc_bb(er_09[i])))

In [45]:
pickle_save((er_09,ans1),"workspace/Synthetic_graph/er_09_MVC_ANS.pkl")

In [4]:
temp1 = pickle_load("workspace/Synthetic_graph/er_01_MVC_ANS.pkl")
temp2 = pickle_load("workspace/Synthetic_graph/er_02_MVC_ANS.pkl")
temp3 = pickle_load("workspace/Synthetic_graph/er_03_MVC_ANS.pkl")
temp4 = pickle_load("workspace/Synthetic_graph/er_04_MVC_ANS.pkl")
temp5 = pickle_load("workspace/Synthetic_graph/er_05_MVC_ANS.pkl")
temp6 = pickle_load("workspace/Synthetic_graph/er_06_MVC_ANS.pkl")
temp7 = pickle_load("workspace/Synthetic_graph/er_07_MVC_ANS.pkl")
temp8 = pickle_load("workspace/Synthetic_graph/er_08_MVC_ANS.pkl")
temp9 = pickle_load("workspace/Synthetic_graph/er_09_MVC_ANS.pkl")

all_graph = temp1[0]+temp2[0]+temp3[0]+temp4[0]+temp5[0]+temp6[0]+temp7[0]+temp8[0]+temp9[0]

all_ans = temp1[1]+temp2[1]+temp3[1]+temp4[1]+temp5[1]+temp6[1]+temp7[1]+temp8[1]+temp9[1]


In [ ]:
from node2vec import Node2Vec
graph_embedding = []
for i in range(len(all_graph)):
    vec = Node2Vec(all_graph[i], dimensions=64, walk_length=30, num_walks=30, workers=4)
    model = vec.fit(window=10, min_count=1, batch_words=4)
    embeddings = model.wv
    graph_info = np.mean([embeddings[node] for node in range(len(embeddings))], axis=0)
    graph_embedding.append(graph_info)
pickle_save((graph_embedding,all_ans),"workspace/Synthetic_graph/embedding_ANS.pkl")

In [6]:
emb, ans = pickle_load("workspace/Synthetic_graph/embedding_ANS.pkl")

In [7]:
print(type(emb[0]))

<class 'numpy.ndarray'>


In [11]:
train_data = []
X = []
Y = []
for i in range(len(emb) -1 ):
    for j in range(i+1 , len(emb)):
        concateemb = np.concatenate((emb[i],emb[j]))
        label = 1 if ans[i] == ans[j] else 0
        train_data.append((concateemb,label))
        X.append(concateemb)
        Y.append(label)

In [10]:
pickle_save(train_data,"workspace/Synthetic_graph/train_data.pkl")

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)


In [ ]:
model = RandomForestClassifier()

# 訓練模型
model.fit(X_train, y_train)

# 預測和評估
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

def graph_to_node_representation(graph):
    num_nodes = graph.number_of_nodes()
    node_features = torch.eye(num_nodes)
    edge_index = torch.tensor(list(graph.edges())).t().contiguous()
    data = Data(x=node_features, edge_index=edge_index)
    return data


In [4]:
from node2vec import Node2Vec
# training_graph = pickle_load("./workspace/Synthetic_graph/Test.pkl")

In [6]:
t = validation_graph_gen(n= 50,p = 0.15, num = 1, graph_type='er')[0]

In [13]:
vec = Node2Vec(t, dimensions=50, walk_length=30, num_walks=30, workers=4)

Computing transition probabilities:   0%|          | 0/50 [00:00<?, ?it/s]

Generating walks (CPU: 4): 100%|██████████| 7/7 [00:00<00:00, 343.33it/s]


In [14]:
model = vec.fit(window=10, min_count=1, batch_words=4)

In [15]:
embeddings = model.wv


In [23]:
print(torch.tensor(embeddings.vectors, dtype=torch.float32).shape)

torch.Size([50, 50])


In [10]:
graph_embedding = np.mean([embeddings[node] for node in range(len(embeddings))], axis=0)


In [11]:
for node in range(5):
    print(f"Node {node}: {embeddings[str(node)]}") 

node_embeddings = {node: embeddings[str(node)] for node in t.nodes()}

# 打印一個節點的嵌入（例如節點 0）
# print("\nNode 0's embedding:", node_embeddings[0])

Node 0: [ 1.35642916e-01 -1.64594024e-01  4.17407304e-02  1.40197262e-01
  1.95480958e-02 -8.24867263e-02  3.24863307e-02  2.28214905e-01
 -8.60718489e-02  2.82436274e-02  8.56611431e-02 -8.44951347e-02
 -9.26422104e-02 -7.94978663e-02  1.39893264e-01  7.51068294e-02
  4.24500480e-02 -8.76105726e-02 -1.52059853e-01  1.64118171e-01
  1.82215236e-02  9.11932290e-02  3.17816377e-01  1.16657212e-01
 -1.62270293e-02  2.61186093e-01 -3.17212082e-02 -1.18185304e-01
 -6.81088120e-02 -1.79164827e-01 -1.20591193e-01  7.97695965e-02
 -2.25098073e-01 -2.68489242e-01 -1.78507373e-01  3.28494425e-05
  1.31850183e-01  8.06379616e-02  3.07244420e-01 -2.01669373e-02
 -1.18654363e-01  7.79571384e-03  1.58096035e-03 -1.16846815e-01
  7.50496015e-02  1.01981265e-02  8.33430793e-03  3.75570208e-02
  1.10275239e-01  3.73766690e-01 -1.22294821e-01 -1.59472749e-02
  3.28983605e-01  3.60854954e-01  6.39303401e-02  3.71168494e-01
  6.45229071e-02 -5.32378815e-02 -4.13615108e-02  2.14883298e-01
  1.36428058e-01 